In [ ]:
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
import itertools
from enum import Enum

print(1 + (3 - 1) % 10)

In [ ]:
def to_grayscale( img: np.ndarray ):
    return 0.3 * img[:, :, 0] + 0.59 * img[:, :, 1] + 0.11 * img[:, :, 2]

In [ ]:
class KernelType(Enum):
    Uniform = 1
    Cross = 2
    CrossDiag = 3

def get_kernel( tp: KernelType, sz: int ):
    halfsz = int(sz / 2)
    if tp == KernelType.Uniform:
        return np.ones((sz, sz))
    elif tp == KernelType.Cross:
        ret = np.zeros((sz, sz))
        ret[halfsz, :] = 1
        ret[:, halfsz] = 1
        return ret
    elif tp == KernelType.CrossDiag:
        ret = np.zeros((sz, sz))
        for i in range(sz):
            ret[i, i] = 1
            ret[i, sz - i - 1] = 1
        return ret

In [ ]:
print(get_kernel(KernelType.CrossDiag, 5))
print()
print(get_kernel(KernelType.Cross, 5))
print()
print(get_kernel(KernelType.Uniform, 5))

In [ ]:
def apply_to_patch(ker: np.ndarray, a2: np.ndarray):
    assert(ker.shape[:2] == a2.shape[:2])
    if len( a2.shape ) == 2:
        a2 = np.expand_dims(a2, -1)

    idx = (ker != 0)

    if a2.shape[-1] == 1:
        return (ker[idx] * a2[idx, 0]).flatten()
    else:
        ret = np.stack([
            ( ker[idx] * a2[idx, i] ).flatten()
            for i in range(a2.shape[-1])
        ], -1)
        return ret

def median_filtering(kernel: np.ndarray, img: np.ndarray):
    ksz = kernel.shape[0]
    halfksz = int(ksz / 2)
    padding = [(halfksz, halfksz), (halfksz, halfksz)]
    if len(img.shape) == 3:
        padding += [(0,0)]
    padded_img = np.pad(img, padding , 'reflect')
    print(padded_img.shape)

    ret = np.zeros_like(img)
    for i, j in itertools.product(range( img.shape[0] ), range( img.shape[1] )):
        ip = i + halfksz
        jp = j + halfksz

        l = ip - halfksz
        r = ip + halfksz
        u = jp - halfksz
        d = jp + halfksz

        ret[i, j] = np.median( apply_to_patch(kernel, padded_img[l:r + 1, u:d + 1]), 0 )

    return ret

In [ ]:
img = np.array(PIL.Image.open( 'data/lena.png' ))
# img = to_grayscale(img)

img_filtered_uni = median_filtering(get_kernel(KernelType.Uniform, 9), img)

plt.subplot(1, 2, 1)
plt.title('Original')
plt.imshow(img)

plt.subplot(1, 2, 2)
plt.title('Uniform')
plt.imshow(img_filtered_uni)

plt.show()

In [ ]:
img = np.array(PIL.Image.open( 'data/noise.png' ))
img = to_grayscale(img)
img[img > 0] = 255

img_filtered_uni = median_filtering(get_kernel(KernelType.Uniform, 15), img)
img_filtered_cross = median_filtering(get_kernel(KernelType.Cross, 15), img)
img_filtered_cross_diag = median_filtering(get_kernel(KernelType.CrossDiag, 15), img)

plt.subplot(1, 5, 1)
plt.title('Original')
plt.imshow(img, cmap='gray')

plt.subplot(1, 5, 2)
plt.title('Uniform')
plt.imshow(img_filtered_uni, cmap='gray')

plt.subplot(1, 5, 3)
plt.title('XOR')
plt.imshow(np.abs( img_filtered_uni - img ), cmap='gray')

plt.subplot(1, 5, 4)
plt.title('Cross')
plt.imshow(img_filtered_cross, cmap='gray')

plt.subplot(1, 5, 5)
plt.title('CrossDiag')
plt.imshow(img_filtered_cross_diag, cmap='gray')

plt.show()